In [1]:
import lightgbm as lgb
import optuna.integration.lightgbm as oplgb
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np

In [2]:
df_Train=pd.read_csv('Train_New.csv')
df_Test=pd.read_csv('Test_New.csv')

In [3]:
df_sample = pd.read_csv('sample submission.csv')

In [4]:
df_Train.drop('PC1', axis=1, inplace=True)
df_Train.drop('PC2', axis=1, inplace=True)
df_Train.drop('PC3', axis=1, inplace=True)
df_Train.drop('PC4', axis=1, inplace=True)
df_Train.drop('PC5', axis=1, inplace=True)
df_Train.drop('Meals', axis=1, inplace=True)

df_Test.drop('PC1', axis=1, inplace=True)
df_Test.drop('PC2', axis=1, inplace=True)
df_Test.drop('PC3', axis=1, inplace=True)
df_Test.drop('PC4', axis=1, inplace=True)
df_Test.drop('PC5', axis=1, inplace=True)
df_Test.drop('Meals', axis=1, inplace=True)


In [5]:
df_Train.drop('N5', axis=1, inplace=True)
df_Test.drop('N5', axis=1, inplace=True)

df_Train.drop('A5', axis=1, inplace=True)
df_Test.drop('A5', axis=1, inplace=True)

df_Train.drop('Month', axis=1, inplace=True)
df_Test.drop('Month', axis=1, inplace=True)

In [6]:
#Adding all features except Target column
feature_cols = [c for c in df_Train.columns if c != "Per Person Price"]

In [7]:
# train_x = df_Train.drop(['Per Person Price'],axis=1)
# train_y = df_Train[['Per Person Price']]
# test_x = df_Test

In [8]:
feature_cols

['Package Name',
 'Package Type',
 'D1',
 'D2',
 'D3',
 'D4',
 'D5',
 'N1',
 'N2',
 'N3',
 'N4',
 'Year',
 'Hotel Details',
 'Start City',
 'A1',
 'A2',
 'A3',
 'A4',
 'Flight Stops',
 'SP1',
 'SP2',
 'SP3',
 'SP4',
 'SP5',
 'SP6',
 'SP7',
 'SP8',
 'SP9',
 'SP10',
 'SP11',
 'SP12',
 'SP13',
 'SP14',
 'SP15',
 'SP16',
 'SP17',
 'SP18',
 'SP19',
 'Cancellation Rules']

In [9]:
train_x = df_Train[feature_cols]
train_y = df_Train['Per Person Price']
test_x = df_Test

In [10]:
train_y

0        11509.0
1        22485.5
2        12421.5
3        35967.0
4        25584.0
          ...   
20995    16168.5
20996    33770.5
20997    29964.0
20998    28145.5
20999    15736.0
Name: Per Person Price, Length: 21000, dtype: float64

In [11]:
folds = KFold(n_splits=10, shuffle=True, random_state=33)

In [12]:
class FoldsAverageLGBM:
    def __init__(self, folds):
        self.folds = folds
        self.models = []
        
    def fit(self, lgb_params, train_x, train_y):
        oof_preds = np.zeros_like(train_y)
        
        self.train_x = train_x.values
        self.train_y = train_y.values
        
        for tr_idx, va_idx in tqdm(folds.split(train_x)):
            tr_x, va_x = self.train_x[tr_idx], self.train_x[va_idx]
            tr_y, va_y = self.train_y[tr_idx], self.train_y[va_idx]
            
            lgb_train_dataset = lgb.Dataset(tr_x, tr_y)
            lgb_valid_dataset = lgb.Dataset(va_x, va_y)
            model = lgb.train(lgb_params, lgb_train_dataset, valid_sets=[lgb_valid_dataset], verbose_eval=100)
            self.models.append(model)
            
            oof_pred = model.predict(va_x)
            oof_preds[va_idx] = oof_pred
            
        self.oof_preds = oof_preds
        
    def predict(self, test_x):
        preds = []
        for model in tqdm(self.models):
            pred = model.predict(test_x)
            preds.append(pred)
        preds = np.mean(preds, axis=0)
        return preds

In [13]:
best_lgb_params = {'objective': 'regression',
'metric': 'rmse',    
'num_leaves' : 662,
'max_bin' : 772,
'learning_rate' : 0.09665690922904038,
'max_depth' : 16,
'min_child_weight' : 16,
'feature_fraction' : 0.5158725304733791,
'bagging_fraction' : 0.9605156946921354,
'bagging_freq' : 2,
'min_child_samples' : 3,
'lambda_l1' : 0.0009722638339694185,
'lambda_l2' : 1.3560862848213769e-05,
}
best_lgb_params["early_stopping_round"] = 5000
best_lgb_params["num_iterations"] = 30000

In [14]:
folds_average_lgbm = FoldsAverageLGBM(folds)

In [15]:
folds_average_lgbm.fit(best_lgb_params, train_x, train_y)

D:\anaconda\lib\site-packages\lightgbm\engine.py:148: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
D:\anaconda\lib\site-packages\lightgbm\engine.py:153: UserWarning: Found `early_stopping_round` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


Training until validation scores don't improve for 5000 rounds
[100]	valid_0's rmse: 5038.59
[200]	valid_0's rmse: 4999.54
[300]	valid_0's rmse: 5046.13
[400]	valid_0's rmse: 5073.62
[500]	valid_0's rmse: 5109.26
[600]	valid_0's rmse: 5136.88
[700]	valid_0's rmse: 5162.26
[800]	valid_0's rmse: 5185.98
[900]	valid_0's rmse: 5199.01
[1000]	valid_0's rmse: 5218.68
[1100]	valid_0's rmse: 5232.42
[1200]	valid_0's rmse: 5247.49
[1300]	valid_0's rmse: 5262.13
[1400]	valid_0's rmse: 5274.65
[1500]	valid_0's rmse: 5283.38
[1600]	valid_0's rmse: 5294.42
[1700]	valid_0's rmse: 5304.24
[1800]	valid_0's rmse: 5313.04
[1900]	valid_0's rmse: 5320.25
[2000]	valid_0's rmse: 5323.89
[2100]	valid_0's rmse: 5328.75
[2200]	valid_0's rmse: 5334.1
[2300]	valid_0's rmse: 5340.17
[2400]	valid_0's rmse: 5342.77
[2500]	valid_0's rmse: 5348.2
[2600]	valid_0's rmse: 5351.72
[2700]	valid_0's rmse: 5355.72
[2800]	valid_0's rmse: 5358.07
[2900]	valid_0's rmse: 5362.15
[3000]	valid_0's rmse: 5367.74
[3100]	valid_0's r

In [19]:
np.sqrt(mean_squared_error(df_Train['Per Person Price'], folds_average_lgbm.oof_preds))

4805.856812052569

In [17]:
y_pred = folds_average_lgbm.predict(test_x)

In [20]:
sub = df_sample.copy()
sub["Per Person Price"] = y_pred

sub.to_csv("WorkathonStarterSubmission_06_V10.csv", index=False)

sub.head(10)

,Per Person Price
0,19278.179092
1,11908.346454
2,5152.266472
3,7353.337042
4,21273.708579
5,48107.885978
6,15892.436081
7,24973.793692
8,53812.812387
9,15896.153782
